In [31]:
# hypatia data
# tidy HIP mess
import pandas as pd

# Read your CSV file into a DataFrame
df = pd.read_csv('hypatia-Raw.csv')  # Replace 'hypatia-Raw.csv' with your actual file name

# Ensure the 'HIP_num' column is treated as strings
df['HIP_num'] = df['HIP_num'].astype(str)

# Keep only rows where 'HIP_num' column starts with 'HIP'
df = df[df['HIP_num'].str.startswith('HIP')].copy()

# Extract the numbers following 'HIP' and create a new column 'HIP'
df['HIP'] = df['HIP_num'].str.extract(r'HIP\s*(\d+)', expand=False)

# Optionally, drop the 'HIP' column if no longer needed
df.drop(columns=['HIP_num'], inplace=True)

# Move 'HIP' column to the first position
columns = df.columns.tolist()
columns.insert(0, columns.pop(columns.index('HIP')))
df = df[columns]

# Optionally, save the updated DataFrame to a new CSV file
df.to_csv('hypatia-ready.csv', index=False)

In [32]:
# NASA Exoplanet Archive
# clean up HIP mess
# remove duplicate HIP
import pandas as pd

# Read your CSV file into a DataFrame
df = pd.read_csv('PlanetarySystems-Raw.csv')  # Replace 'hypatia-Raw.csv' with your actual file name

# Ensure the 'HIP_num' column is treated as strings
df['HIP_num'] = df['HIP_num'].astype(str)

# Keep only rows where 'HIP_num' column starts with 'HIP'
df = df[df['HIP_num'].str.startswith('HIP')].copy()

# Extract the numbers following 'HIP' and create a new column 'HIP'
df['HIP'] = df['HIP_num'].str.extract(r'HIP\s*(\d+)', expand=False)

# Optionally, drop the 'HIP' column if no longer needed
df.drop(columns=['HIP_num'], inplace=True)

# Move 'HIP' column to the first position
columns = df.columns.tolist()
columns.insert(0, columns.pop(columns.index('HIP')))
df = df[columns]

# **Remove duplicate HIP entries**
df.drop_duplicates(subset=['HIP'], inplace=True)

# Optionally, save the updated DataFrame to a new CSV file
df.to_csv('PlanetarySystems-ready.csv', index=False)

In [35]:
# combine hypatia and NASA into one
# filter for thin
# filter out Imaging and Transit
# fill in zeroes
import pandas as pd

# Read the two CSV files into DataFrames
df1 = pd.read_csv('hypatia-ready.csv')  # First dataset
df2 = pd.read_csv('PlanetarySystems-ready.csv')  # Second dataset

# Ensure the 'HIP' columns are of the same data type
df1['HIP'] = df1['HIP'].astype(int)
df2['HIP'] = df2['HIP'].astype(int)

# Merge the two DataFrames on the 'HIP' column
merged_df = pd.merge(df1, df2, on='HIP', how='outer')

# Remove rows where 'discoverymethod' is 'Imaging' or 'Transit'
merged_df = merged_df[~merged_df['discoverymethod'].isin(['Imaging', 'Transit'])].copy()

# Retain only rows where 'f_disk' column is 'thin ' (with trailing space)
merged_df = merged_df[merged_df['f_disk'] == 'thin '].copy()

# Optionally, save the merged DataFrame to a new CSV file
merged_df.to_csv('combined-dataset.csv', index=False)

In [40]:
import pandas as pd
import numpy as np  # Import numpy for NaN representation

# Read the two CSV files into DataFrames
df1 = pd.read_csv('hypatia-ready.csv')  # First dataset
df2 = pd.read_csv('PlanetarySystems-ready.csv')  # Second dataset

# Ensure the 'HIP' columns are of the same data type
df1['HIP'] = df1['HIP'].astype(int)
df2['HIP'] = df2['HIP'].astype(int)

# Merge the two DataFrames on the 'HIP' column
merged_df = pd.merge(df1, df2, on='HIP', how='outer')

# Remove rows where 'discoverymethod' is 'Imaging' or 'Transit'
merged_df = merged_df[~merged_df['discoverymethod'].isin(['Imaging', 'Transit'])].copy()

# Remove rows where 'f_disk' is 'thick'
merged_df = merged_df[~merged_df['f_disk'].isin(['thick '])].copy()

# Fill in '0' for empty entries in 'Multi' and 'Exo' columns
merged_df['Multi'] = merged_df['Multi'].replace('', np.nan).fillna(0)
merged_df['Exo'] = merged_df['Exo'].replace('', np.nan).fillna(0)
merged_df['MaxPMass'] = merged_df['MaxPMass'].replace('', np.nan).fillna(0)
merged_df['StarMass'] = merged_df['StarMass'].replace('', np.nan).fillna(0)

# Fill in 'None' for empty entries in 'discoverymethod' column
merged_df['discoverymethod'] = merged_df['discoverymethod'].replace('', np.nan).fillna('None')

# Reset the index
merged_df.reset_index(drop=True, inplace=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('combined-dataset.csv', index=False)
